In [1]:
import sys
import torch

seed = 42
#torch.set_default_dtype(torch.float64)
torch.manual_seed(seed)
torch.set_printoptions(precision=2, sci_mode=False)


import pandas as pd
import numpy as np
import ast
import copy

# Load Modules
sys.path.append("..")
from os.path import expanduser
home = expanduser("~")
from fiora.MOL.constants import DEFAULT_PPM, PPM, DEFAULT_MODES
from fiora.IO.LibraryLoader import LibraryLoader
from fiora.MOL.FragmentationTree import FragmentationTree 
import fiora.visualization.spectrum_visualizer as sv

from sklearn.metrics import r2_score
import scipy
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

print(f'Working with Python {sys.version}')


[10:52:36] WARNING: not removing hydrogen atom without neighbors
[10:52:36] WARNING: not removing hydrogen atom without neighbors
[10:52:36] WARNING: not removing hydrogen atom without neighbors


Working with Python 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]


## Data

In [2]:
from typing import Literal
lib: Literal["NIST", "MSDIAL", "NIST/MSDIAL", "MSnLib"] = "MSnLib" #"MSnLib"
print(f"Preparing {lib} library")

debug_mode = False # Default: False
if debug_mode:
    print("+++ This is a test run (debug mode) with a small subset of data points. Results are not representative. +++")

Preparing MSnLib library


In [3]:
# key map to read metadata from pandas DataFrame
metadata_key_map = {
                "name": "Name",
                "collision_energy":  "CE", 
                "instrument": "Instrument_type",
                "ionization": "Ionization",
                "precursor_mz": "PrecursorMZ",
                "precursor_mode": "Precursor_type",
                "retention_time": "RETENTIONTIME",
                "ccs": "CCS"
                }


#
# Load specified libraries and align metadata
#

def load_training_data():
    if ("NIST" in lib or "MSDIAL" in lib):
        data_path: str = f"{home}/data/metabolites/preprocessed/datasplits_Jan24.csv"
    elif lib == "MSnLib":
        data_path: str = f"{home}/data/metabolites/preprocessed/datasplits_msnlib_April25_v1.csv"
    else:
        raise NameError(f"Unknown library selected {lib=}.")
    L = LibraryLoader()
    df = L.load_from_csv(data_path)
    return df

df = load_training_data()

# Restore dictionary values
dict_columns = ["peaks", "summary"]
for col in dict_columns:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x.replace('nan', 'None')))
    #df[col] = df[col].apply(ast.literal_eval)
    
df['group_id'] = df['group_id'].astype(int)


In [4]:
# import polars as pl
# pdf = pl.from_pandas(df)

In [5]:
%%capture
from fiora.MOL.Metabolite import Metabolite
from fiora.GNN.AtomFeatureEncoder import AtomFeatureEncoder
from fiora.GNN.BondFeatureEncoder import BondFeatureEncoder
from fiora.GNN.SetupFeatureEncoder import SetupFeatureEncoder


CE_upper_limit = 100.0
weight_upper_limit = 1000.0


if debug_mode:
    df = df.iloc[:10000,:]
    #df = df.iloc[5000:20000,:]

overwrite_setup_features = None
if lib == "MSnLib":
    overwrite_setup_features = {
        "instrument": ["HCD"],
        "precursor_mode": ["[M+H]+", "[M-H]-", "[M]+", "[M]-"]
    
    }


df["Metabolite"] = df["SMILES"].apply(Metabolite)
df["Metabolite"].apply(lambda x: x.create_molecular_structure_graph())

node_encoder = AtomFeatureEncoder(feature_list=["symbol", "num_hydrogen", "ring_type"])
bond_encoder = BondFeatureEncoder(feature_list=["bond_type", "ring_type"])
setup_encoder = SetupFeatureEncoder(feature_list=["collision_energy", "molecular_weight", "precursor_mode", "instrument"], sets_overwrite=overwrite_setup_features)
rt_encoder = SetupFeatureEncoder(feature_list=["molecular_weight", "precursor_mode", "instrument"], sets_overwrite=overwrite_setup_features)

setup_encoder.normalize_features["collision_energy"]["max"] = CE_upper_limit 
setup_encoder.normalize_features["molecular_weight"]["max"] = weight_upper_limit 
rt_encoder.normalize_features["molecular_weight"]["max"] = weight_upper_limit 

df["Metabolite"].apply(lambda x: x.compute_graph_attributes(node_encoder, bond_encoder))
df.apply(lambda x: x["Metabolite"].set_id(x["group_id"]) , axis=1)

#df["summary"] = df.apply(lambda x: {key: x[name] for key, name in metadata_key_map.items()}, axis=1)
df.apply(lambda x: x["Metabolite"].add_metadata(x["summary"], setup_encoder, rt_encoder), axis=1)
df.apply(lambda x: x["Metabolite"].set_loss_weight(x["loss_weight"]), axis=1)

In [6]:
from fiora.MOL.MetaboliteIndex import MetaboliteIndex

mindex: MetaboliteIndex = MetaboliteIndex()
mindex.index_metabolites(df["Metabolite"])

In [ ]:
mindex.create_fragmentation_trees()
list_of_mismatched_ids = mindex.add_fragmentation_trees_to_metabolite_list(df["Metabolite"], graph_mismatch_policy="recompute")
print(f"Total number of recomputed trees: {len(list_of_mismatched_ids)}")

In [ ]:
%%capture
#df["Metabolite"].apply(lambda x: x.fragment_MOL(depth=1))
df.apply(lambda x: x["Metabolite"].match_fragments_to_peaks(x["peaks"]["mz"], x["peaks"]["intensity"], tolerance=x["ppm_peak_tolerance"]), axis=1)

In [ ]:
df["num_peak_matches"] = df["Metabolite"].apply(lambda x: x.match_stats["num_peak_matches"])
print(sum(df["num_peak_matches"] < 1))
df = df[df["num_peak_matches"] >= 2]

0


##### Load Casmi data

In [ ]:
casmi16_path = f"{home}/data/metabolites/CASMI_2016/casmi16_withCCS.csv"
casmi22_path = f"{home}/data/metabolites/CASMI_2022/casmi22_withCCS.csv"

df_cas = pd.read_csv(casmi16_path, index_col=[0], low_memory=False)
df_cas22 = pd.read_csv(casmi22_path, index_col=[0], low_memory=False)

# Restore dictionary values
dict_columns = ["peaks", "Candidates"]
for col in dict_columns:
    df_cas[col] = df_cas[col].apply(ast.literal_eval)

df_cas22["peaks"] = df_cas22["peaks"].apply(ast.literal_eval)

In [ ]:
%%capture
from fiora.MOL.collision_energy import NCE_to_eV

df_cas["RETENTIONTIME"] = df_cas["RTINSECONDS"] / 60.0
df_cas["Metabolite"] = df_cas["SMILES"].apply(Metabolite)
df_cas["Metabolite"].apply(lambda x: x.create_molecular_structure_graph())

df_cas["Metabolite"].apply(lambda x: x.compute_graph_attributes(node_encoder, bond_encoder))
df_cas["CE"] = 20.0 # actually stepped 20/35/50
df_cas["Instrument_type"] = "HCD" # CHECK if correct Orbitrap

metadata_key_map16 = {"collision_energy":  "CE", 
                 "instrument": "Instrument_type",
                 "precursor_mz": "PRECURSOR_MZ",
                 'precursor_mode': "Precursor_type",
                 "retention_time": "RETENTIONTIME"
                 }

df_cas["summary"] = df_cas.apply(lambda x: {key: x[name] for key, name in metadata_key_map16.items()}, axis=1)
df_cas.apply(lambda x: x["Metabolite"].add_metadata(x["summary"], setup_encoder), axis=1)

# Fragmentation
df_cas["Metabolite"].apply(lambda x: x.fragment_MOL(depth=1))
df_cas.apply(lambda x: x["Metabolite"].match_fragments_to_peaks(x["peaks"]["mz"], x["peaks"]["intensity"], tolerance=100 * PPM), axis=1) # Optional: use mz_cut instead

#
# CASMI 22
#

df_cas22["Metabolite"] = df_cas22["SMILES"].apply(Metabolite)
df_cas22["Metabolite"].apply(lambda x: x.create_molecular_structure_graph())

df_cas22["Metabolite"].apply(lambda x: x.compute_graph_attributes(node_encoder, bond_encoder))
df_cas22["CE"] = df_cas22.apply(lambda x: NCE_to_eV(x["NCE"], x["precursor_mz"]), axis=1)

metadata_key_map22 = {"collision_energy":  "CE", 
                 "instrument": "Instrument_type",
                 "precursor_mz": "precursor_mz",
                 'precursor_mode': "Precursor_type",
                 "retention_time": "ChallengeRT"
                 }

df_cas22["summary"] = df_cas22.apply(lambda x: {key: x[name] for key, name in metadata_key_map22.items()}, axis=1)
df_cas22.apply(lambda x: x["Metabolite"].add_metadata(x["summary"], setup_encoder, rt_encoder), axis=1)

# Fragmentation
df_cas22["Metabolite"].apply(lambda x: x.fragment_MOL(depth=1))
df_cas22.apply(lambda x: x["Metabolite"].match_fragments_to_peaks(x["peaks"]["mz"], x["peaks"]["intensity"], tolerance=100 * PPM), axis=1) # Optional: use mz_cut instead

df_cas22 = df_cas22.reset_index()

## Model Training and Testing

In [ ]:
import torch_geometric as geom

if torch.cuda.is_available(): 
    torch.cuda.empty_cache()
    dev = "cuda:0"
else: 
    dev = "cpu" 
 
print(f"Running on device: {dev}")




Running on device: cuda:0


##### Load Data

In [ ]:
print(df.groupby("dataset")["group_id"].unique().apply(len))

df_test = df[df["dataset"] == "test"]
df_train = df[df["dataset"].isin(["training", "validation"])]

dataset
test           1799
training      14392
validation     1799
Name: group_id, dtype: int64


In [ ]:
geo_data = df_train["Metabolite"].apply(lambda x: x.as_geometric_data().to(dev)).values
print(f"Prepared training/validation with {len(geo_data)} data points")

Prepared training/validation with 63144 data points


##### Prepare Model

In [ ]:
model_params = {
    'param_tag': 'default',
    'gnn_type': 'RGCNConv',
    'depth': 6,
    'hidden_dimension': 300,
    'dense_layers': 2,
    'embedding_aggregation': 'concat',
    'embedding_dimension': 300,
    'input_dropout': 0.2,
    'latent_dropout': 0.1,
    'node_feature_layout': node_encoder.feature_numbers,
    'edge_feature_layout': bond_encoder.feature_numbers,    
    'static_feature_dimension': geo_data[0]["static_edge_features"].shape[1],
    'static_rt_feature_dimension': geo_data[0]["static_rt_features"].shape[1],
    'output_dimension': len(DEFAULT_MODES) * 2, # per edge 
    
    # Keep track of encoded features
    'atom_features': node_encoder.feature_list,
    'atom_features': bond_encoder.feature_list,
    'setup_features': setup_encoder.feature_list,
    'setup_features_categorical_set': setup_encoder.categorical_sets,
    'rt_features': rt_encoder.feature_list,
    
    # Set default flags (May be overwritten below)
    'rt_supported': False,
    'ccs_supported': False,
    'version': "x.x.x"
    
}
training_params = {
    'epochs': 200 if not debug_mode else 10, 
    'batch_size': 256,
    #'train_val_split': 0.90,
    'learning_rate': 0.0004, # 0.00001 currently for wMAE # Default for wMSE is 0.0004, #0.001,
    'weight_decay': 1e-4,
    'with_RT': False, # Turn off RT/CCS for initial trainings round
    'with_CCS': False
}

##### Begin Training

In [ ]:
import numpy as np

# Subsample training data
def subsample_keys(train_keys, val_keys, down_to_fraction: float):
    train_sample = np.random.choice(train_keys, size=int(len(train_keys) * down_to_fraction), replace=False)
    val_sample = np.random.choice(val_keys, size=int(len(val_keys) * down_to_fraction), replace=False)
    return train_sample, val_sample


In [ ]:
from fiora.GNN.SpectralTrainer import SpectralTrainer
from fiora.GNN.GNNModules import GNNCompiler
from fiora.GNN.Losses import WeightedMSELoss, WeightedMSEMetric, WeightedMAELoss, WeightedMAEMetric
from fiora.MS.SimulationFramework import SimulationFramework

fiora = SimulationFramework(None, dev=dev, with_RT=True, with_CCS=True)
# fiora = SimulationFramework(None, dev=dev, with_RT=training_params["with_RT"], with_CCS=training_params["with_CCS"])
np.seterr(invalid='ignore')
tag = "training"
val_interval = 1
metric_dict= {"mse": WeightedMSEMetric} #WeightedMSEMetric
loss_fn = WeightedMSELoss() # WeightedMSELoss()
all_together = False
down_sample = False

if all_together:
    val_interval = 200
    metric_dict=None
    loss_fn = torch.nn.MSELoss()    

def train_new_model(continue_with_model=None):
    if continue_with_model:
        model = continue_with_model.to(dev)
    else:
        model = GNNCompiler(model_params).to(dev)
        
    y_label = 'compiled_probsSQRT' # y_label = 'compiled_probsALL'
    optimizer = torch.optim.Adam(model.parameters(), lr=training_params["learning_rate"], weight_decay=training_params["weight_decay"])
    if all_together:
        trainer = SpectralTrainer(geo_data, y_tag=y_label, problem_type="regression", only_training=True, metric_dict=metric_dict, split_by_group=True, seed=seed, device=dev)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
    else:
        train_keys, val_keys = df[df["dataset"] == "training"]["group_id"].unique(), df[df["dataset"] == "validation"]["group_id"].unique()
        if down_sample:
            train_fraction = 0.10
            train_keys, val_keys = subsample_keys(train_keys, val_keys, train_fraction) # Downsample training data for test
            print(f"Sample down to {train_fraction * 100}% with {len(train_keys)} training and {len(val_keys)} validation compounds ")
        trainer = SpectralTrainer(geo_data, y_tag=y_label, problem_type="regression", train_keys=train_keys, val_keys=val_keys, metric_dict=metric_dict, split_by_group=True, seed=seed, device=dev)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 8, factor=0.5, mode = 'min', verbose = True)

    
    checkpoints = trainer.train(model, optimizer, loss_fn, scheduler=scheduler, batch_size=training_params['batch_size'], epochs=training_params["epochs"], val_every_n_epochs=1, with_CCS=training_params["with_CCS"], with_RT=training_params["with_RT"], use_validation_mask=False, tag=tag) #, mask_name="compiled_validation_maskALL")   
    print(checkpoints)
    return model, checkpoints

def simulate_all(model, DF):
    return fiora.simulate_all(DF, model)

    
def test_model(model, DF, score="spectral_sqrt_cosine", return_df=False):
    dft = simulate_all(model, DF)
    
    if return_df:
        return dft
    return dft[score].values

##### Test CASMI 16 and 22

In [ ]:
from fiora.MOL.collision_energy import NCE_to_eV
from fiora.MS.spectral_scores import spectral_cosine, spectral_reflection_cosine, reweighted_dot
from fiora.MS.ms_utility import merge_annotated_spectrum

def test_cas16(model, df_cas=df_cas, score="merged_sqrt_cosine", return_df=False):
    
    df_cas["NCE"] = 20.0 # actually stepped NCE 20/35/50
    df_cas["CE"] = df_cas[["NCE", "PRECURSOR_MZ"]].apply(lambda x: NCE_to_eV(x["NCE"], x["PRECURSOR_MZ"]), axis=1)
    df_cas["step1_CE"] = df_cas["CE"]
    df_cas["summary"] = df_cas.apply(lambda x: {key: x[name] for key, name in metadata_key_map16.items()}, axis=1)
    df_cas.apply(lambda x: x["Metabolite"].add_metadata(x["summary"], setup_encoder, rt_encoder), axis=1)
    df_cas = fiora.simulate_all(df_cas, model, suffix="_20")

    df_cas["NCE"] = 35.0 # actually stepped NCE 20/35/50
    df_cas["CE"] = df_cas[["NCE", "PRECURSOR_MZ"]].apply(lambda x: NCE_to_eV(x["NCE"], x["PRECURSOR_MZ"]), axis=1)
    df_cas["step2_CE"] = df_cas["CE"]
    df_cas["summary"] = df_cas.apply(lambda x: {key: x[name] for key, name in metadata_key_map16.items()}, axis=1)
    df_cas.apply(lambda x: x["Metabolite"].add_metadata(x["summary"], setup_encoder, rt_encoder), axis=1)
    df_cas = fiora.simulate_all(df_cas, model, suffix="_35")


    df_cas["NCE"] = 50.0 # actually stepped NCE 20/35/50
    df_cas["CE"] = df_cas[["NCE", "PRECURSOR_MZ"]].apply(lambda x: NCE_to_eV(x["NCE"], x["PRECURSOR_MZ"]), axis=1)
    df_cas["step3_CE"] = df_cas["CE"]
    df_cas["summary"] = df_cas.apply(lambda x: {key: x[name] for key, name in metadata_key_map16.items()}, axis=1)
    df_cas.apply(lambda x: x["Metabolite"].add_metadata(x["summary"], setup_encoder, rt_encoder), axis=1)
    df_cas = fiora.simulate_all(df_cas, model, suffix="_50")

    df_cas["avg_CE"] = (df_cas["step1_CE"] + df_cas["step2_CE"] + df_cas["step3_CE"]) / 3

    df_cas["merged_peaks"] = df_cas.apply(lambda x: merge_annotated_spectrum(merge_annotated_spectrum(x["sim_peaks_20"], x["sim_peaks_35"]), x["sim_peaks_50"]) , axis=1)
    df_cas["merged_cosine"] = df_cas.apply(lambda x: spectral_cosine(x["peaks"], x["merged_peaks"]), axis=1)
    df_cas["merged_sqrt_cosine"] = df_cas.apply(lambda x: spectral_cosine(x["peaks"], x["merged_peaks"], transform=np.sqrt), axis=1)
    df_cas["merged_sqrt_cosine_wo_prec"] = df_cas.apply(lambda x: spectral_cosine(x["peaks"], x["merged_peaks"], transform=np.sqrt, remove_mz=x["Metabolite"].get_theoretical_precursor_mz(x["Metabolite"].metadata["precursor_mode"])), axis=1)
    df_cas["merged_refl_cosine"] = df_cas.apply(lambda x: spectral_reflection_cosine(x["peaks"], x["merged_peaks"], transform=np.sqrt), axis=1)
    df_cas["merged_steins"] = df_cas.apply(lambda x: reweighted_dot(x["peaks"], x["merged_peaks"]), axis=1)
    df_cas["spectral_sqrt_cosine"] = df_cas["merged_sqrt_cosine"] # just remember it is merged
    df_cas["spectral_sqrt_cosine_wo_prec"] = df_cas["merged_sqrt_cosine_wo_prec"] # just remember it is merged

    df_cas["coverage"] = df_cas["Metabolite"].apply(lambda x: x.match_stats["coverage"])
    df_cas["RT_pred"] = df_cas["RT_pred_35"]
    df_cas["RT_dif"] = df_cas["RT_dif_35"]
    df_cas["CCS_pred"] = df_cas["CCS_pred_35"]
    df_cas["library"] = "CASMI-16"
    
    if return_df:
        return df_cas
    
    return df_cas[score].values

In [ ]:
#raise KeyboardInterrupt("Stop before starting a new training run!")

In [ ]:
#model = GNNCompiler(model_params).to(dev)
#model.to("cpu")
# device = next(model.parameters()).device
# device


In [ ]:
print(f"Training model")
model, checkpoints = train_new_model() # continue_with_model=model)

Training model
Using pre-set train/validation keys


/home/ynowatzk/miniforge3/envs/fiora/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


	Validation RMSE: 0.008289
	 >> Set new checkpoint to epoch 1
	Validation RMSE: 0.008082
	 >> Set new checkpoint to epoch 2
	Validation RMSE: 0.007879
	 >> Set new checkpoint to epoch 3
	Validation RMSE: 0.007276
	 >> Set new checkpoint to epoch 4
	Validation RMSE: 0.007073
	 >> Set new checkpoint to epoch 5
	Validation RMSE: 0.007071
	 >> Set new checkpoint to epoch 6
	Validation RMSE: 0.006969
	 >> Set new checkpoint to epoch 7
	Validation RMSE: 0.006968
	Validation RMSE: 0.006867
	 >> Set new checkpoint to epoch 9
	Validation RMSE: 0.0067066
	 >> Set new checkpoint to epoch 10
	Validation RMSE: 0.0067065
	Validation RMSE: 0.0066064
	 >> Set new checkpoint to epoch 12
	Validation RMSE: 0.0066063
	 >> Set new checkpoint to epoch 13
	Validation RMSE: 0.0065062
	 >> Set new checkpoint to epoch 14
	Validation RMSE: 0.0065061
	Validation RMSE: 0.0065061
	 >> Set new checkpoint to epoch 16
	Validation RMSE: 0.0064060
	 >> Set new checkpoint to epoch 17
	Validation RMSE: 0.0065059
	Validati

In [ ]:
import copy

print(checkpoints) 
print(np.sqrt(checkpoints["val_loss"]))
model_at_last_epoch = copy.deepcopy(model)

{'epoch': 156, 'val_loss': 3.6315686884336174e-05, 'file': '../../checkpoint_training.best.pt'}
0.006026249819277008


In [ ]:
model = GNNCompiler.load(checkpoints["file"]).to(dev)
score = "spectral_sqrt_cosine"

val_results = test_model(model, df_train[df_train["dataset"]== "validation"], score=score)
test_results = test_model(model, df_test, score=score)
casmi16_results = test_cas16(model, score=score)
casmi16_p = test_cas16(model, df_cas[df_cas["Precursor_type"] == "[M+H]+"], score=score)
casmi16_n = test_cas16(model, df_cas[df_cas["Precursor_type"] == "[M-H]-"], score=score)
casmi22_results = test_model(model, df_cas22, score=score)
casmi22_p = test_model(model, df_cas22[df_cas22["Precursor_type"] == "[M+H]+"], score=score)
casmi22_n = test_model(model, df_cas22[df_cas22["Precursor_type"] == "[M-H]-"], score=score)
    
results = [{"model": model, "validation": val_results, "test": test_results, "casmi16": casmi16_results, "casmi22": casmi22_results, "casmi16+": casmi16_p, "casmi16-": casmi16_n, "casmi22+": casmi22_p, "casmi22-": casmi22_n}]

/tmp/ipykernel_1780528/3303304107.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cas["NCE"] = 20.0 # actually stepped NCE 20/35/50
/tmp/ipykernel_1780528/3303304107.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cas["CE"] = df_cas[["NCE", "PRECURSOR_MZ"]].apply(lambda x: NCE_to_eV(x["NCE"], x["PRECURSOR_MZ"]), axis=1)
/tmp/ipykernel_1780528/3303304107.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [ ]:
score = "spectral_sqrt_cosine_wo_prec"
val_results = test_model(model, df_train[df_train["dataset"]== "validation"], score=score)
test_results = test_model(model, df_test, score=score)
casmi16_results = test_cas16(model, score=score)
casmi16_p = test_cas16(model, df_cas[df_cas["Precursor_type"] == "[M+H]+"], score=score)
casmi16_n = test_cas16(model, df_cas[df_cas["Precursor_type"] == "[M-H]-"], score=score)
casmi22_results = test_model(model, df_cas22, score=score)
casmi22_p = test_model(model, df_cas22[df_cas22["Precursor_type"] == "[M+H]+"], score=score)
casmi22_n = test_model(model, df_cas22[df_cas22["Precursor_type"] == "[M-H]-"], score=score)
    
results_wop = [{"model": model, "validation": val_results, "test": test_results, "casmi16": casmi16_results, "casmi22": casmi22_results, "casmi16+": casmi16_p, "casmi16-": casmi16_n, "casmi22+": casmi22_p, "casmi22-": casmi22_n}]

/tmp/ipykernel_1780528/3303304107.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cas["NCE"] = 20.0 # actually stepped NCE 20/35/50
/tmp/ipykernel_1780528/3303304107.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cas["CE"] = df_cas[["NCE", "PRECURSOR_MZ"]].apply(lambda x: NCE_to_eV(x["NCE"], x["PRECURSOR_MZ"]), axis=1)
/tmp/ipykernel_1780528/3303304107.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [ ]:
LOG = pd.DataFrame(results)
eval_columns = LOG.columns[1:].fillna(0.0)
LOG[eval_columns] = LOG[eval_columns].apply(lambda x: x.apply(np.median))

LOG

,model,validation,test,casmi16,casmi22,casmi16+,casmi16-,casmi22+,casmi22-
0,GNNCompiler(\n (node_embedding): FeatureEmbed...,0.789238,0.738194,0.677314,0.25998,0.658437,0.730632,0.253072,0.265494


In [ ]:
LOG = pd.DataFrame(results_wop).fillna(0.0)
eval_columns = LOG.columns[1:]
LOG[eval_columns] = LOG[eval_columns].apply(lambda x: x.apply(np.nan_to_num).apply(np.median))

LOG

,model,validation,test,casmi16,casmi22,casmi16+,casmi16-,casmi22+,casmi22-
0,GNNCompiler(\n (node_embedding): FeatureEmbed...,0.755023,0.691917,0.380801,0.230422,0.364927,0.426074,0.217827,0.262897


In [ ]:
model.model_params

{'param_tag': 'default',
 'gnn_type': 'RGCNConv',
 'depth': 6,
 'hidden_dimension': 300,
 'dense_layers': 2,
 'embedding_aggregation': 'concat',
 'embedding_dimension': 300,
 'input_dropout': 0.2,
 'latent_dropout': 0.1,
 'node_feature_layout': {'symbol': 10, 'num_hydrogen': 5, 'ring_type': 5},
 'edge_feature_layout': {'bond_type': 4, 'ring_type': 5},
 'static_feature_dimension': 8,
 'static_rt_feature_dimension': 7,
 'output_dimension': 10,
 'atom_features': ['bond_type', 'ring_type'],
 'setup_features': ['collision_energy',
  'molecular_weight',
  'precursor_mode',
  'instrument'],
 'setup_features_categorical_set': {'instrument': ['HCD'],
  'precursor_mode': ['[M+H]+', '[M-H]-', '[M]+', '[M]-']},
 'rt_features': ['molecular_weight', 'precursor_mode', 'instrument'],
 'rt_supported': False,
 'ccs_supported': False,
 'version': 'x.x.x',
 'training_label': 'compiled_probsSQRT'}

In [ ]:
# LOG = pd.DataFrame(results)
# eval_columns = LOG.columns[3:]

# home_path = f"{home}/data/metabolites/benchmarking/"
# NAME = model_params["gnn_type"] + "_depth.csv"
# for col in eval_columns:
#     LOG[col] = LOG[col].apply(lambda x: str(list(x)))
# LOG.to_csv(home_path + NAME, index=False, sep="\t")

In [ ]:
# LOGIC = pd.read_csv(home_path + NAME, sep="\t")
# for col in eval_columns:
#     LOGIC[col] = LOGIC[col].apply(lambda x: ast.literal_eval(x.replace('nan', 'None')))

# LOGIC[eval_columns] = LOGIC[eval_columns].apply(lambda x: x.apply(np.median))
# LOGIC

In [ ]:
raise KeyboardInterrupt("Halt! Make sure you wish to save/overwrite model files")

KeyboardInterrupt: Halt! Make sure you wish to save/overwrite model files

In [ ]:
depth = model_params["depth"]
MODEL_PATH = f"{home}/data/metabolites/pretrained_models/v0.1.1_OS_depth{depth}_April25_vXXX.pt"
model.save(MODEL_PATH)
print(f"Saved to {MODEL_PATH}")

Saved to /home/ynowatzk/data/metabolites/pretrained_models/v0.1.1_OS_depth6_April25_v2.pt


In [ ]:
raise KeyboardInterrupt()

KeyboardInterrupt: 

In [ ]:
from fiora.GNN.GNNModules import GNNCompiler

In [ ]:
df_test["Metabolite"].iloc[0].as_geometric_data()

Data(x=[32, 3], edge_index=[2, 72], edge_attr=[72, 2], y=[72, 1], edge_type=[72], static_graph_features=[1, 8], static_edge_features=[72, 8], static_rt_features=[1, 7], compiled_probs=[74], compiled_probs2=[146], compiled_probs6=[434], compiled_probsALL=[722], compiled_probsSQRT=[722], compiled_probsDEPRE=[722], compiled_probs_wo_prec=[722], compiled_counts=[74], edge_break_count=[72, 1], retention_time=[1, 1], retention_mask=[1, 1], ccs=[1, 1], ccs_mask=[1, 1], validation_mask=[72, 1], compiled_validation_mask=[74], compiled_validation_mask2=[146], compiled_validation_mask6=[434], compiled_validation_maskALL=[722], group_id=11, weight=[1, 1], weight_tensor=[722], is_node_aromatic=[32, 1], is_edge_aromatic=[72, 1])

In [ ]:
dev="cuda:0"
mymy = GNNCompiler.load(MODEL_PATH) # f"{home}/data/metabolites/pretrained_models/v0.0.1_merged_2.pt"
#mymy.load_state_dict(torch.load(f"{home}/data/metabolites/pretrained_models/test.pt"))
mymy.eval()
mymy = mymy.to(dev)


In [ ]:
np.median(test_model(mymy, df_test))

0.6777992132153692

In [ ]:
np.median(test_model(mymy, df_cas22))

0.2357750642225984

In [ ]:
import json
with open(MODEL_PATH.replace(".pt", "_params.json"), 'r') as fp:
    p = json.load(fp)
hh = GNNCompiler(p)
hh.load_state_dict(torch.load(MODEL_PATH.replace(".pt", "_state.pt")))
hh.eval()
hh = hh.to(dev)

In [ ]:
np.median(test_model(hh, df_cas22))

0.2357750642225984

In [ ]:
raise KeyboardInterrupt("TODO")

KeyboardInterrupt: TODO

In [ ]:
## prepare output for for CFM-ID
import os
save_df = False
cfm_directory = f"{home}/data/metabolites/cfm-id/"
name = "test_split_negative_solutions_cfm.txt"
df_cfm = df_test[["group_id", "SMILES", "Precursor_type"]]
df_n = df_cfm[df_cfm["Precursor_type"] == "[M-H]-"].drop_duplicates(subset='group_id', keep='first')
df_p = df_cfm[df_cfm["Precursor_type"] == "[M+H]+"].drop_duplicates(subset='group_id', keep='first')

print(df_n.head())

if save_df:
    file = os.path.join(cfm_directory, name)
    df_n[["group_id", "SMILES"]].to_csv(file, index=False, header=False, sep=" ")
    
    name = name.replace("negative", "positive")
    file = os.path.join(cfm_directory, name)
    df_p[["group_id", "SMILES"]].to_csv(file, index=False, header=False, sep=" ")

       group_id                                             SMILES  \
91358       695  CCCCCCCCCCCCCCCC(=O)OC[C@@H](O)COP(=O)([O-])OC...   
91888      1028  CN1c2c([nH]c(N)nc2=O)NCC1CNc1ccc(C(=O)N[C@@H](...   
91898      1030  CC(C)C[C@@H](N)[C@H](O)C(=O)N[C@H](C(=O)N[C@H]...   
92031      8617        CCCCC[C@H](O)/C=C/C=C\C/C=C\C/C=C\CCCC(=O)O   
92174      1061                    CC(=O)N[C@@H](CCCNC(=N)N)C(=O)O   

      Precursor_type  
91358         [M-H]-  
91888         [M-H]-  
91898         [M-H]-  
92031         [M-H]-  
92174         [M-H]-  
